In [ ]:
import mne
import mne_bids
from mne_bids import BIDSPath
import os, os.path as op
import nilearn
import matplotlib.pyplot as plt
import numpy as np
from mne.minimum_norm import apply_inverse, make_inverse_operator
import glob

from nilearn import *

from nih2mne.utilities.bids_helpers import get_mri_dict
n_jobs = 10 #Number of parrallel operations

## Set some generic BIDS information

In [ ]:
#Raw data
bids_root = op.join('/data/', os.environ['USER'], 'meg_workshop_data')  
#Processed Data Folder: contains (freesurfer / Day1 / Day2) 
deriv_root = op.join(bids_root, 'derivatives')  
#This course data - bids derivatives outputs for Day2 - hey thats today
project_root = op.join(deriv_root, 'Day2')  
#Freesurfer brain surface reconstruction
fs_subjects_dir = subjects_dir = op.join(deriv_root, 'freesurfer','subjects')

### Find data for a subject

In [ ]:
subject = 'ON02811'
data_dict = get_mri_dict(subject,bids_root, task='airpuff', project='Day2')

#Not in BIDs format
stc_fname = op.join(project_root, 'ses-01', 'meg', 'sub-ON02811_ses-01_task-airpuff_run-01_desc-missingstim_stc30to50-vl.stc')

# get_mri_dict is a helper function that provides all of the loaders for the mri integration
# Each dictionary item codes for the specific loader of that file - see use in next cell of code
data_dict

#### Load the preprocessed data - Items from Lab3 MRI Processing

In [ ]:
bem = data_dict['bem'].load()
fwd = data_dict['fwd'].load()
volfwd = data_dict['volfwd'].load()
src = fwd['src']
volsrc = volfwd['src']
trans = data_dict['trans'].load()

In [ ]:
#Load the Raw data
bids_path = BIDSPath(root=bids_root, subject=subject, task='airpuff', run='01',session='01')
raw = mne.io.read_raw_ctf(bids_path.fpath, clean_names=True, preload=True, verbose=False)
raw.filter(0.3, 110, n_jobs=n_jobs)

### Prepare Imaging for Source Localization

In [ ]:
volfwd

In [ ]:
fs_volsrc_fname=op.join(project_root, 'fsave-src.fif')
fs_volsrc = mne.read_source_spaces(fs_volsrc_fname)

morph = mne.compute_source_morph(volfwd['src'], subject_from = 'sub-'+subject, 
                                 subject_to='fsaverage', subjects_dir=fs_subjects_dir, src_to=fs_volsrc)

### Load up one of the volume source estimate files for subject ON02811

In [ ]:
stc_fname = op.join(project_root, 'sub-ON02811', 'ses-01', 'sub-ON02811_ses-01_task-airpuff_run-01_desc-stim_stc30to50-vl.stc')
stc=mne.read_source_estimate(stc_fname)
stc

In [ ]:
stc_fsaverage = morph.apply(stc)

In [ ]:
_ = stc.plot(volsrc, subject='sub-'+subject, subjects_dir = fs_subjects_dir)

In [ ]:
fs_volsrc_fname=op.join(project_root, 'fsave-src.fif')
fs_volsrc = mne.read_source_spaces(fs_volsrc_fname)

In [ ]:
_ = stc_fsaverage.plot(fs_volsrc_fname, subject='fsaverage', subjects_dir = fs_subjects_dir)

In [ ]:
stc_list

In [ ]:
stc_list = glob.glob(op.join(project_root, 'sub-*', 'ses-01', 'sub-*_ses-01_task-airpuff_run-01_desc-stim_stc30to50-vl.stc'))

In [ ]:
volfwd

In [ ]:
stc_fsave_list = []
for stc_fname in stc_list:
    subject = op.basename(stc_fname).split('_')[0][4:]
    data_dict = get_mri_dict(subject,bids_root, task='airpuff', project='Day2')
    stc = mne.read_source_estimate(stc_fname)
    volfwd=data_dict['volfwd'].load()
    morph = mne.compute_source_morph(volfwd['src'], subject_from = 'sub-'+subject, 
                                 subject_to='fsaverage', subjects_dir=fs_subjects_dir, src_to=fs_volsrc)
    stc_fsave_list.append( morph.apply(stc ))
    
    

In [ ]:
#Convert to numpy array
group_dat = np.stack([stc_.data for stc_ in stc_fsave_list])

## Don't do statistics this way -- just a quick way to visualize

In [ ]:
import copy
stc_allav = copy.copy(stc_fsaverage)
#PUt the average of the numpy array into the new data
stc_allav._data = group_dat.mean(axis=0)

In [ ]:
_ = stc_allav.plot(fs_volsrc_fname, subject='fsaverage', subjects_dir = fs_subjects_dir)

## Parcellation

In [ ]:
labels=mne.read_labels_from_annot('sub-'+subject,parc='aparc', 
                                   subjects_dir=subjects_dir)        




In [ ]:
stc_surf_fname = op.join(project_root, 'TEST-src.fif')

In [ ]:
stc_surf = mne.read_source_estimate(stc_surf_fname)

In [ ]:
ts = mne.extract_label_time_course([stc_surf], labels, fwd['src'])[0]

## TS is a list (if epochs are entered) - We only have the average data here

In [ ]:
ts

In [ ]:
ts.shape

In [ ]:
len(labels)

### Because these are large parcels - the activity in the somatosensory cortex is reduced / averaged over teh rest of the parcel

In [ ]:
plt.plot(stc_surf.times, ts.T)